In [6]:
import pathlib
import pygame
from collections import defaultdict

def split_code(code):
    """ lav koden om til hvordan brikken ser ud

    positive tal betyder overkrop
    negative betyder underkrop

    to sider passer sammen hvis summen er nul
    
    """
    for q in [3,2,1,0]:
        r = 10**q
        M = code // r
        code -= M*r
        sign = 1 if q>1 else -1
        yield sign*M






class Brik:
    _colors = {
        1:"blå",
        2:"lilla",
        3:"grøn",
        4:"brun",
    }
    _width = 200

    def __init__(self, code):
        """ det her sker når vi laver en ny brik
        """
        self._path = pathlib.Path("img") / f"{code}.jpg"
        self._sides = [color for color in split_code(code)]
        self.rotation = 0
        self._image = None
        self._sprite = None

    @property
    def sides(self):       
        s = [ f"{self._colors[abs(_side)]}-{'top' if _side>0 else 'bund'}"  for _side in self._sides ]

        for piruet in range(self.rotation):
            s = s[1:] + [s[0]]

        return s

    def __str__(self):
        return f"{self._path}: {self.sides}"
        
    def __repr__(self):
        return str(self)
    
    def drej(self, piruetter=1):
        """ roter med uret
                    2
        rot=0     1   3      
                    0

                    1
        rot=1     0   2      
                    3

                    
        """
        self.rotation += piruetter
        self.rotation = self.rotation % 4

    @property
    def down(self):
        """
        get whatever is on the left

        r=0  : 0
        r=1  : 3
        r=2  : 2
        r=3  : 1
        
        """
        return self._sides[(-self.rotation + 0) % 4]
    @property
    def left(self):
        return self._sides[(-self.rotation + 1) % 4]
    @property
    def up(self):
        return self._sides[(-self.rotation + 2) % 4]
    @property
    def right(self):
        return self._sides[(-self.rotation + 3) % 4]


    def match(self, down=None, left=None, up=None, right=None):

        if down and self.down + down.up != 0:
            return False
        if left and self.left + left.right != 0:
            return False
        if up and self.up + up.down != 0:
            return False
        if right and self.right + right.left != 0:
            return False
    
        return True
    
    @classmethod
    def from_path(cls, path):
        code, *_ = path.stem.split("#")
        brik = cls(int(code))
        brik._path = path
        return brik

    def add_sprite(self):
        self._image = pygame.image.load(self._path).convert()  # Load image
        self._image = pygame.transform.scale(self._image, (self._width, self._width))

        
        self._sprite = pygame.sprite.Sprite()  # Create a sprite object
        self._sprite.image = self._image  # Set the sprite's image
        self._sprite.rect = self._image.get_rect()  # Get the image's rectangle


    def set_xy(self, x, y):
        """ 
            (0,0)   (1, 0) ...
            (0,1)   (1, 1) ...
            ...
        
        """

        self._sprite.rect.x = self._width * x
        self._sprite.rect.y = self._width * y


    def increment_x(self):
        self._sprite.rect.x += self._width
    def increment_y(self):
        self._sprite.rect.y += self._width
    def decrement_x(self):
        self._sprite.rect.x -= self._width
    def decrement_y(self):
        self._sprite.rect.y -= self._width



    def apply_rotation(self):

        rotated_image = pygame.transform.rotate(self._image, -90 * self.rotation)
        self._sprite.image = rotated_image


    @classmethod
    def from_folder(cls, folder):
        brikker = []
        
        for path in folder.glob("*.jpg"):
            try:
                brik = Brik.from_path(path)
                brikker.append(brik)
            except:
                pass
        return brikker


In [7]:


def pre_match(current, others):

    possible_matches = defaultdict(list)
    
    for own_rotation in range(4):
        current.rotation = own_rotation
        
        for other in others:
            if other._path == current._path:
                continue

            for other_rotation in range(4):
                other.rotation = other_rotation
                if current.match(down=other):
                    possible_matches[own_rotation, "down"].append((other._path, other_rotation))
                if current.match(left=other):
                    possible_matches[own_rotation, "left"].append((other._path, other_rotation))
                if current.match(up=other):
                    possible_matches[own_rotation, "up"].append((other._path, other_rotation))
                if current.match(right=other):
                    possible_matches[own_rotation, "right"].append((other._path, other_rotation))
                
    return possible_matches

from tqdm.notebook import tqdm


def all_first_pieces(brikker):
    return { (brik._path, rotation) for brik in brikker for rotation in range(4)}


class Board:

    config = [
        (0, 0, "down", "right"),
        (0, 1, "down", "up", "right"),
        (0, 2, "down", "up", "right"),
        (0, 3, "up", "right"),
        (1, 0, "down", "left", "right"),
        (1, 1, "down", "left", "up", "right"),
        (1, 2, "down", "left", "up", "right"),
        (1, 3, "left", "up", "right"),
        (2, 0, "down", "left", "right"),
        (2, 1, "down", "left", "up", "right"),
        (2, 2, "down", "left", "up", "right"),
        (2, 3, "left", "up", "right"),
        (3, 0, "down", "left"),
        (3, 1, "down", "left", "up"),
        (3, 2, "down", "left", "up"),
        (3, 3, "left", "up"),
    ]

    """
    0 4 8  12
    1 5 9  13
    2 6 10 14
    3 7 11 15
    """
    
    next_match = [
        [],                          #0 has no constraints
        [(0, "down")],               #1 must be in the down matches of 0
        [(1, "down")],               #2 must be in the down matches of 1
        [(2, "down")],               #3 must be in the down matches of 2
        [(0, "right")],              #4 must be in the right matches of 0
        [(1, "right"), (4, "down")], #5
        [(2, "right"), (5, "down")], #6
        [(3, "right"), (6, "down")], #7
        [(4, "right")],              #8 must be in the right matches of 0
        [(5, "right"), (8, "down")], #9
        [(6, "right"), (9, "down")], #10
        [(7, "right"), (10, "down")], #11
        [(8, "right")],               #12 must be in the right matches of 0
        [(9, "right"), (12, "down")], #13
        [(10, "right"), (13, "down")], #14
        [(11, "right"), (14, "down")], #15
    ]

    skips = {
        "down":1,
        "left":-4,
        "up":-1,
        "right":4,
    }
    
    def __init__(self, brikker):
        

        self._all = all_first_pieces(brikker)
        self.registry = {brik._path:brik for brik in brikker}

        for brik in brikker:
            brik.pre_match = pre_match(brik, brikker)

        self.reset()
    
    def reset(self):
        self.used = set()
        self.placements = [None for _ in range(16)]
        self.i = 0
    
    def check(self):

        for idx, (brik, (_, _, *neighbors)) in enumerate(zip(self.placements, self.config)):
            if brik is None:
                continue

            neighbors_to_check = {direction:self.placements[idx + self.skips[direction]] for direction in neighbors}
            is_match = brik.match(**neighbors_to_check)

            if not is_match:
                return False
        return True
    
    def place(self, path, rotation):
        brik = self.registry[path]
        brik.rotation = rotation
        self.placements[self.i] = brik
        
        for rot in range(4):
            self.used.add((brik._path, rot))
        self.i += 1


    def next(self):
        """yield all possibilities:

        not already used
        in the possible matches for neighbors
        
        """
        pre_matches = self._all.difference(self.used)
        for idx_to_match, direction_to_match in self.next_match[self.i]:
            other = self.placements[idx_to_match]
            # a list in the shape [(path, rotation), ...]
            pre_matches = pre_matches.intersection(other.pre_match[ other.rotation, direction_to_match ])

        yield from pre_matches
            

    def set_board(self, placements):
        self.reset()
        for pth, rot in placements:
            self.place(pth, rot)
    

In [8]:
def read_config(ls):
    configs = [l.strip().split(",") for l in ls]
    configs = [ [c.split(":") for c in cl] for cl in configs ]
    configs = [ [(pathlib.Path(p), int(r)) for (p,r) in cl] for cl in configs ]
    return configs

def write_config(board):
    return ",".join([f"{brik._path}:{brik.rotation}" for brik in board.placements[:board.i]]) + "\n"


brikker = Brik.from_folder(pathlib.Path("img"))
board = Board(brikker)

In [122]:
with open("1.txt", "w") as f:
    for p,r in tqdm(board.next()):
        board.set_board([(p,r)])
        valid = board.check()

        if valid:
            f.write(write_config(board))
    

0it [00:00, ?it/s]

In [4]:

def next(i):
    
    with open(f"{i-1}.txt", "r") as f:
        ls = f.readlines()
    
    configs =  read_config(ls)
    
    pb = tqdm()
    
    nye_spil = 0
    with open(f"{i}.txt", "w") as f:
        for config in configs:
            board.set_board(config)
            for p,r in board.next():
                board.set_board(config + [(p,r)])
                valid = board.check()
        
                if valid:
                    f.write(write_config(board))
                    nye_spil += 1
    
                if nye_spil > 1000:
                    pb.update(nye_spil)
                    pb.display()
                    nye_spil = 0
    
    pb.update(nye_spil)
    pb.display()
    nye_spil = 0

for i in range(2, 17):
    next(i)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

200

In [ ]:
with open(f"16.txt", "r") as f:
    ls = f.readlines()

configs =  read_config(ls)


pygame.init()

# Define window dimensions
width, height = Brik._width * 4, Brik._width * 4
active_overlay_color = (0, 255, 0, 50) # 0 is transparent and 255 is opaque

error_overlay_color = (255, 0, 0, 64) # 0 is transparent and 255 is opaque

# Create a Pygame window
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Professor spillet")


board.set_board(configs[4])
brikker = board.placements

for i, brik in enumerate(brikker):
    brik.add_sprite()
    brik.set_xy(i // 4, i%4)
    brik.apply_rotation()
    # You can set initial positions here if needed

active_brick_index = len(brikker) - 1  # Initialize with the last brick

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        elif event.type == pygame.KEYDOWN:
            current_brik = brikker[active_brick_index]
            
            if event.key == pygame.K_w:
                # Move the active brick up
                current_brik.decrement_y()
            elif event.key == pygame.K_s:
                # Move the active brick down
                current_brik.increment_y()
            elif event.key == pygame.K_a:
                # Move the active brick left
                current_brik.decrement_x()
            elif event.key == pygame.K_d:
                # Move the active brick right
                current_brik.increment_x()
            elif event.key == pygame.K_q:
                # Rotate the active brick counterclockwise
                current_brik.drej(-1)
                current_brik.apply_rotation()
            elif event.key == pygame.K_e:
                # Rotate the active brick clockwise
                current_brik.drej(1)
                current_brik.apply_rotation()

            elif event.key == pygame.K_TAB:
                # Switch to the next active brick (Tab key)
                active_brick_index = (active_brick_index + 1) % len(brikker)

    # Ensure the active brick index stays within valid bounds
    active_brick_index = max(0, min(active_brick_index, len(brikker) - 1))

    
    # Clear the screen
    screen.fill((255, 255, 255))

    # Blit (draw) each sprite onto the screen
    for i, brik in enumerate(brikker):
        screen.blit( brik._sprite.image, brik._sprite.rect)

        if i == active_brick_index and False:
            overlay_surface = pygame.Surface((brik._sprite.rect.width, brik._sprite.rect.height), pygame.SRCALPHA)
            overlay_surface.fill(active_overlay_color)
            screen.blit(overlay_surface, brik._sprite.rect)

    
    pygame.display.flip()

pygame.quit()
